## Deliverable 2. Create a Customer Travel Destinations Map.

In [23]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [24]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/weather_data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Meadow Lake,US,34.8014,-106.5436,63.61,29,21,7.70,few clouds
1,1,Thompson,CA,55.7435,-97.8558,7.65,91,100,9.57,overcast clouds
2,2,Nelson Bay,AU,-32.7167,152.1500,68.00,97,76,4.32,broken clouds
3,3,Skuodas,LT,56.2667,21.5333,30.06,86,100,9.93,overcast clouds
4,4,Bredasdorp,ZA,-34.5322,20.0403,62.28,70,93,11.86,overcast clouds


In [25]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 70


In [26]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_weather_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
city_weather_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Meadow Lake,US,34.8014,-106.5436,63.61,29,21,7.70,few clouds
2,2,Nelson Bay,AU,-32.7167,152.1500,68.00,97,76,4.32,broken clouds
4,4,Bredasdorp,ZA,-34.5322,20.0403,62.28,70,93,11.86,overcast clouds
15,15,Victoria,HK,22.2855,114.1577,62.58,45,71,1.01,broken clouds
16,16,Port Macquarie,AU,-31.4333,152.9167,66.16,100,90,5.75,overcast clouds
22,22,Cape Town,ZA,-33.9258,18.4232,66.34,60,88,15.21,overcast clouds
29,29,Lebu,CL,-37.6167,-73.6500,60.28,62,100,7.36,overcast clouds
30,30,Mar Del Plata,AR,-38.0023,-57.5575,60.87,80,98,15.99,overcast clouds
31,31,Airai,TL,-8.9266,125.4092,61.72,88,99,1.07,moderate rain
40,40,Hobart,AU,-42.8794,147.3294,60.03,84,90,0.00,overcast clouds


In [27]:
# 4a. Determine if there are any empty rows.
city_weather_df.isnull().sum()
city_weather_df.count()

City_ID                101
City                   101
Country                 99
Lat                    101
Lng                    101
Max Temp               101
Humidity               101
Cloudiness             101
Wind Speed             101
Weather Description    101
dtype: int64

In [28]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_weather_df.dropna()

In [29]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Weather Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

    except (IndexError):
        print("Hotel not found... skipping.") 
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [31]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Weather Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [32]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df.dropna()

In [33]:
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Weather Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [34]:
clean_hotel_df.tail(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
597,Mangai,CD,69.31,overcast clouds,-4.0500,19.5333,
599,Talcahuano,CL,64.08,clear sky,-36.7167,-73.1167,departamento comodo
602,Monywa,MM,68.05,few clouds,22.1167,95.1333,Monywa Hotel
607,Najran,SA,63.25,clear sky,17.4924,44.1277,محمد هادي ال فطيح
626,Marawi,PH,67.64,overcast clouds,7.9986,124.2928,Derogongan Residence
631,Sapulpa,US,67.91,clear sky,35.9987,-96.1142,Super 8 by Wyndham Sapulpa/Tulsa Area
636,Comodoro Rivadavia,AR,69.66,clear sky,-45.8667,-67.5000,Austral Hotel
645,Bardiyah,LY,62.02,overcast clouds,31.7561,25.0865,بورتو بردية
654,Moree,AU,66.22,clear sky,-29.4667,149.8500,Molika Springs Motel
666,Byron Bay,AU,69.87,overcast clouds,-28.6500,153.6167,Backpackers Inn


In [35]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [36]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))